# Import packages

In [199]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
sns.set_theme()

# Dataset and variable explanation

In [200]:
df = pd.read_csv('data.csv', encoding= 'unicode_escape')

In [201]:
df.head()

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,...,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,"17,50,000",acquired,USA,NY,New York City,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,"40,00,000",operating,USA,CA,Los Angeles,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Publishing,"40,000",operating,EST,NaN,Tallinn,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,/organization/in-touch-network,(In)Touch Network,http://www.InTouchNetwork.com,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,"15,00,000",operating,GBR,NaN,London,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,"60,000",operating,USA,TX,Dallas,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [202]:
df = df.rename({' funding_total_usd ': 'funding_total_usd', ' market ': 'market'}, axis='columns')

## Columns description

permalink - link in crunchbase api

name - startup name

homepage_url - website

category_list - list of market categories

market - market direction

funding_total_usd - total funds in $

status - startup's working status

funding_rounds - number of the rounds

seed - first official funding stage

venture - capital invested in a project in which there is a substantial element of risk, typically a new or expanding business.

equity_crowdfunding - involves exchanging relatively small amounts of cash allowing investors to own a proportionate 
slice of equity in the business.

undisclosed - investment amount not shared publicitly 

convertible_note - a technique that seed investors use to invest in early-stage businesses whose valuation is not yet determined. It is a short-term debt structured to a start-up investment that isn't ready to get evaluated yet. Once the company is valued, Convertible Note is changed into equity.

debt_financing - is a type of funding provided to startups by an investor or lender, such as a bank, for a certain amount of time. 

angel - angel investors are wealthy private investors focused on financing small business ventures in exchange for equity.

private equity - is an alternative investment class that invests in or acquires private companies that are not listed on a public stock exchange

IPO – An initial public offering or stock launch is a public offering in which shares of a company are sold to institutional investors 

post_ipo_equity - a post IPO equity round takes place when firms invest in a company after the company has already gone public. 

post_ipo_debt - a post IPO debt round is a transaction when corporates loan a company money after the company has already gone public.


product_crowdfunding - crowdfunding is a way of raising money to finance projects 

rounds A-H - different rounds of funding

# Cleanup

## Size

In [203]:
print('(Rows, Columns)', df.shape)

(Rows, Columns) (54294, 39)


## Variables

### Categorical

In [204]:
CATEGORICAL = list(filter(lambda x: df[x].dtype=='object',df.columns))
def show_stats(columns):
    stat = {}
    for col in columns:
        stat[col] =  [df[col].nunique(), df[col].isna().mean()*100, df[col].dtype]
    return pd.DataFrame.from_dict(stat, orient='index', columns=['Unique',  'Missed values in %', 'DType'])
show_stats(CATEGORICAL)

,Unique,Missed values in %,DType
permalink,49436,8.943898,object
name,49350,8.945740,object
homepage_url,45850,15.296350,object
category_list,16675,16.239363,object
market,753,16.252256,object
funding_total_usd,14617,8.943898,object
status,3,11.364055,object
country_code,115,18.655837,object
state_code,61,44.448742,object
region,1089,18.655837,object


State code column can be dropped as it doesn't really play a big role, but has too much missed values
We can remove everything connected with links, as there are too many unique values for them. Also remove category list, because we already have Market column

In [205]:
df = df.drop('state_code', axis=1)
df = df.drop(['permalink', 'homepage_url', 'category_list'], axis=1)

funding_total_usd should be converted from str to int

In [206]:
df['funding_total_usd'].head(2)

0     17,50,000 
1     40,00,000 
Name: funding_total_usd, dtype: object

In [207]:
def str_to_int(s: str):
    try:
        s = s.replace(',', '')
        if '-' in s:
            s = 0
        return int(s)
    except ValueError:
        return s
    except AttributeError:
        return s
df['funding_total_usd'] = df['funding_total_usd'].apply(str_to_int)

In [208]:
df = df[df['funding_total_usd']!=0]

### Numeric

In [209]:
show_stats(list(filter(lambda x: df[x].dtype=='float64',df.columns)))

,Unique,Missed values in %,DType
funding_total_usd,14616,10.611192,float64
funding_rounds,17,10.611192,float64
founded_year,101,29.779516,float64
seed,3337,10.611192,float64
venture,9300,10.611192,float64
equity_crowdfunding,252,10.611192,float64
undisclosed,687,10.611192,float64
convertible_note,299,10.611192,float64
debt_financing,1872,10.611192,float64
angel,999,10.611192,float64


## Drop nans and duplicates

In [210]:
df = df.dropna()
df = df.drop_duplicates()
print('Length after cleanup', len(df))

Length after cleanup 27970


# Decode countries
Add the column 'country': USA -> United States

In [211]:
!pip install pycountry
import pycountry

def code_to_country(text: str):
    if text == 'ROM':
        text = 'ROU'
    return pycountry.countries.get(alpha_3=text).name

df['country'] = df['country_code'].apply(code_to_country)
df.drop('country_code', axis=1, inplace=True)

In [212]:
df.columns

Index(['name', 'market', 'funding_total_usd', 'status', 'region', 'city',
       'funding_rounds', 'founded_at', 'founded_month', 'founded_quarter',
       'founded_year', 'first_funding_at', 'last_funding_at', 'seed',
       'venture', 'equity_crowdfunding', 'undisclosed', 'convertible_note',
       'debt_financing', 'angel', 'grant', 'private_equity', 'post_ipo_equity',
       'post_ipo_debt', 'secondary_market', 'product_crowdfunding', 'round_A',
       'round_B', 'round_C', 'round_D', 'round_E', 'round_F', 'round_G',
       'round_H', 'country'],
      dtype='object')

# Vizual analysis

In [213]:
top_spheres = df['market'].value_counts()[:10]
fig = px.pie(values=top_spheres, names=top_spheres.index, title='Top 10 most expensive markets');
fig.show()

In [214]:
fig = px.histogram(df['status'], title='Startups status')
fig.show()

As we can see, most of the startups are operating

In [215]:

rounds = {}
for i in df.columns:
    if 'round_' in i:
        rounds[i] = df[i].mean()/10**6
fig = px.histogram(x=rounds.keys(), y=rounds.values(), labels={'y': 'Investment ($ million)', 'x': 'Rounds'});
fig.show()

Each round you get less and less money – because you can make profit yourself!

In [216]:
fig = px.histogram(df, x='founded_year', labels={'founded_year': 'Founded year'}, title='Startups founded years distribution')
fig.show()

Most of the startups were founded after 2000

In [217]:
fig = px.histogram(x=['angel', 'grant', 'venture'], y=[len(df[df['angel']!=0]),len(df[df['grant']!=0]), len(df[df['venture']!=0])], title='Startups investment sources');
fig.show()

This is obvious, because venture capitals are provided by professional investors and give startups way more funding

In [218]:
top_countries = df['country'].value_counts()[:10]
fig = px.pie(values=top_countries, names=top_countries.index, title='Top 10 most popular regions for a startup');
fig.show()

Hmmm, will the funding be also biggest in USA?

In [219]:
funds_by_country = {}
for c in set(df['country']):
    funds_by_country[c] = df[df['country']==c]['funding_total_usd'].sum()/10**9

funds_by_country = sorted(funds_by_country.items(), key=lambda x: x[1], reverse=True)

fig = px.histogram(
    y=[x[0] for x in funds_by_country][:10], 
    x=[x[1] for x in funds_by_country][:10], 
    labels={'x': 'Total fund ($ billions)', 'y': 'Country'},
    title='Top 10 countries by total funding');
fig.show()

It turns out – yes

And how about companies?

In [220]:
top_companies = df.sort_values(by=['funding_total_usd'], ascending=False)[:25]

fig = px.histogram(data_frame=top_companies, x='funding_total_usd', 
y='name', 
title='Top 25 most successfull companies',
labels={'x': 'Total funding', 'y': 'Company'}, 
color='founded_year');
fig.show()

## Correlation

In [221]:
fig = px.imshow(df.corr(), title='Correlation heatmap');
fig.show();

/var/folders/rx/0vqj6d315l59zn8g17g69ysw0000gn/T/ipykernel_1966/2986737727.py:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



### Question: The plot shows high positive correlation between **debt_financing** and **funding_total_usd**. Why so?

In [222]:
fig = px.scatter(df[(df['debt_financing']>0)&(df['funding_total_usd']/10**9 < 30)], 
x='funding_total_usd', 
y='debt_financing', 
trendline="ols",
title='Correlation between debt financing and startup funding')
fig.show()

Now it's clear that the more money you need for the startup, the more debt_financing you'll need.

### Hypothesis: every year startups need more $$$ to raise. Is that true?
Let's calculate total funding by year and make a line plot

In [223]:
years = set(df['founded_year'])
fund_by_year = {}
for year in sorted(years):
    fund_by_year[year] = df[df['founded_year']==year]['funding_total_usd'].sum()/10**9
fig = px.line(x = fund_by_year.keys(), y=fund_by_year.values(), labels={'x': 'Year', 'y': 'Funding in $ billions'})
fig.show()


From this graph, economic recessions can clearly be seen. In the 1983, Israel bank stock crisis (https://en.wikipedia.org/wiki/1983_Israel_bank_stock_crisis) hit the market and the banks no longer had the capital to buy back shares and to support the prices causing share prices to collapse. Then, from 1984 to 2007, startups raised more and more $$$ each year, untill the Global Financial Crisis in 2007 (https://en.wikipedia.org/wiki/Financial_crisis_of_2007–2008), the most serious in the 21st century. 
So, the Hypothesis is partly true. Indeed, stratups raise more and more capital each year, but it strongly depends on the global events like crises. 

### Hypothesis: your startup will less likely close at less popular market
See dependency between **closed** and **market** variables

In [224]:
markets = {}
ind = 0
for market in set(df['market']):
    closed = len(df[(df['market']==market)&(df['status']=='closed')])
    markets[f"{market}"] = [closed, len(df[df['market']==market])/len(df)*100]
    ind += 1
markets = pd.DataFrame.from_dict(markets, orient='index', columns=[ 'closed', 'share'])
markets = markets.sort_values(by='closed', ascending=False)
markets.to_csv('markets.csv')

In [225]:
fig = px.imshow(
    markets[['share']][:20],
    aspect='auto',
    title='Top 20 markets where startups close',
    labels=dict(x="Market share", y="Markets", color="Market share (%)"),
)
fig.show()

The hypothesis turns out to be true! The closer market to the top, the more 'yellow' its market share becomes. 
Thus, choose less popular markets to succeed!

### Hypothesis: the **seed** you raise depends on the **founded quater**
Are there any 'good' quaters to start your first fund raising?

In [226]:
df['founded_quarter'].head()

0     2012-Q2
2     2012-Q4
3     2011-Q2
4     2014-Q1
10    2012-Q1
Name: founded_quarter, dtype: object

Extract quater into separate column first

In [227]:
def separate_quater(text: str):
    return text.split('-')[-1]

df['quater'] = df['founded_quarter'].apply(separate_quater)
df.to_csv('cleaned_data.csv')

In [228]:
fig = px.scatter(
    y=df.sort_values(by='quater')['seed'], 
    x=df.sort_values(by='quater')['quater'],
    title='Seed values for various quaters',
    labels={'x': 'Quater the startup was founded at', 
    'y': 'Seed size'})
fig.show()

In [229]:
df_by_q_sort = df.sort_values(by='quater')
num_seed = []
for q in df_by_q_sort['quater'].unique():
    num_seed.append(len(df_by_q_sort[(df_by_q_sort['quater']==q)&(df_by_q_sort['seed'] > 0)]))
fig = px.histogram(
    y=num_seed, 
    x=df_by_q_sort['quater'].unique(),
    title='Quater x Seed Number plot',
    labels={'x': 'Quater the startup was founded at', 
    'y': 'Number of seeds raised'})
fig.show()

Unfortunately, no. Seed capital seems to depend on other factors. 
However, **the biggest number** of seeds raised was at the **1st Quater**! That's the most popular time startups raise their first money